In [1]:
import os
from smolagents import AzureOpenAIServerModel, TransformersModel

MODE = 'azure'  # or 'azure'
if MODE == 'azure':
    model = AzureOpenAIServerModel(
        model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
        api_version=os.environ.get("OPENAI_API_VERSION", "")
    )
if MODE == 'local':
    model = TransformersModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
        device="cuda" if os.environ.get("CUDA_VISIBLE_DEVICES", "") else "cpu",
        max_length=512,
        temperature=0.7,
        top_p=0.9,
        top_k=50
    )

In [2]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "pre_medications").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [3]:
from medminer.task.medication import medication_task

for doc in docs[:1]:
    medication_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: medication                                                                                           │
│ Prompt:                                                                                                         │
│     Given a list of medications, save all medications for the patient as csv.                                   │
│     To complete the task make the following steps:                                                              │
│     1. extract all information defined in the columns below from the document. Infer the                        │
│ medication_name_corrected column.                                                                               │
│     2. get the RXCUI for all medications. Use the corrected name of the medications.                            │
│     3. get the ATC code for all medications. Use the rxcui of the medications.                                  │
│     4. save the medication information as csv with the columns defined below.                                   │
│                                                                                                                 │
│     Columns:                                                                                                    │
│     - patient_id: The patient ID.                                                                               │
│     - medication_name: The name of the medication in the document without dose, unit or additional information. │
│     - medication_name_corrected: Use the following format "Brand name or medication name (active ingredient)".  │
│ e.g. "Aspirin (acetylsalicylic acid)".                                                                          │
│     - dose: The dose of the medication. this sould only contain the numeric value.                              │
│     - unit: The unit of the dose (e.g. ml, mg, ...). if not applicable, write an empty string.                  │
│     - dosage_morning: The dose in the morning. if not applicable, write a 0.                                    │
│     - dosage_noon: The dose in the noon. if not applicable, write a 0.                                          │
│     - dosage_evening: The dose in the evening. if not applicable, write a 0.                                    │
│     - dosage_night: The dose in the night. if not applicable, write a 0.                                        │
│     - dosage_information: Additional information about the dosage. if not applicable, write an empty string.    │
│     - rxcui: The RXCUI of the medication. if not applicable, write an empty string.                             │
│     - atc_id: The ATC code of the medication. if not applicable, write an empty string.                         │
│     - atc_name: The name of the ATC code. if not applicable, write an empty string.                             │
│     - atc_type: The type of the ATC code. if not applicable, write an empty string.                             │
│                                                                                                                 │
│                                                                                                                 │
│ --------------------------------------------------------------------------------                                │
│ Data:                                                                                                           │
│     Patiend: 4                                                                                                  │
│     "                                                                                                           │
│     Pantoprazol 40 mg  0-0-1                          

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'extract_medication_data' with arguments: {'data': [{'patient_id': 4, 'medication_name':          │
│ 'Pantoprazol', 'medication_name_corrected': 'Pantoprazol (pantoprazole)', 'dose': 40, 'unit': 'mg',             │
│ 'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 1, 'dosage_night': 0, 'dosage_information': ''},       │
│ {'patient_id': 4, 'medication_name': 'HCT', 'medication_name_corrected': 'HCT (hydrochlorothiazide)', 'dose':   │
│ 25, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0,                │
│ 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'Bisoprolol', 'medication_name_corrected':      │
│ 'Bisoprolol (bisoprolol)', 'dose': 5, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, │
│ 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'Ramipril',                  │
│ 'medication_name_corrected': 'Ramipril (ramipril)', 'dose': 5, 'unit': 'mg', 'dosage_morning': 1,               │
│ 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4,          │
│ 'medication_name': 'Torasemid', 'medication_name_corrected': 'Torasemid (torasemide)', 'dose': 10, 'unit':      │
│ 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, │
│ {'patient_id': 4, 'medication_name': 'Clopidogrel', 'medication_name_corrected': 'Clopidogrel (clopidogrel)',   │
│ 'dose': 75, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0,        │
│ 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'ASS', 'medication_name_corrected': 'ASS        │
│ (acetylsalicylic acid)', 'dose': 100, 'unit': '', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0,   │
│ 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'Macumar',                   │
│ 'medication_name_corrected': 'Macumar (phenprocoumon)', 'dose': 0, 'unit': '', 'dosage_morning': 0,             │
│ 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': 'nach INR'}]}                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'patient_id': 4, 'medication_name': 'Pantoprazol', 'medication_name_corrected': 'Pantoprazol 
(pantoprazole)', 'dose': 40, 'unit': 'mg', 'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 1, 
'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'HCT', 
'medication_name_corrected': 'HCT (hydrochlorothiazide)', 'dose': 25, 'unit': 'mg', 'dosage_morning': 1, 
'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 
'medication_name': 'Bisoprolol', 'medication_name_corrected': 'Bisoprolol (bisoprolol)', 'dose': 5, 'unit': 'mg', 
'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, 
{'patient_id': 4, 'medication_name': 'Ramipril', 'medication_name_corrected': 'Ramipril (ramipril)', 'dose': 5, 
'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': 
''}, {'patient_id': 4, 'medication_name': 'Torasemid', 'medication_name_corrected': 'Torasemid (torasemide)', 
'dose': 10, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 
'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'Clopidogrel', 'medication_name_corrected': 
'Clopidogrel (clopidogrel)', 'dose': 75, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 
'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 'medication_name': 'ASS', 
'medication_name_corrected': 'ASS (acetylsalicylic acid)', 'dose': 100, 'unit': '', 'dosage_morning': 1, 
'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 4, 
'medication_name': 'Macumar', 'medication_name_corrected': 'Macumar (phenprocoumon)', 'dose': 0, 'unit': '', 
'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': 'nach INR'}]

[Step 1: Duration 10.69 seconds| Input tokens: 2,043 | Output tokens: 582]

...

[Step 4: Duration 21.50 seconds| Input tokens: 14,148 | Output tokens: 1,597]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The medication information for patient 4 has been      │
│ successfully saved as a CSV file at /workspaces/examples/result/medication.csv.'}                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The medication information for patient 4 has been successfully saved as a CSV file at 
/workspaces/examples/result/medication.csv.

[Step 5: Duration 47.44 seconds| Input tokens: 19,801 | Output tokens: 1,635]